In [1]:
import pandas as pd
import pymysql
import re,json,os

In [5]:
base = os.environ['BUG_FREE_EUREKA_BASE']
pathTofile = os.path.join(base,'data','exercise06','E-TABM-713_A-MEXP-1551-analytics.tsv')


with open(os.path.join(base,'secrets.json')) as f:
    secrets = json.load(f)

In [6]:
conn = pymysql.connect(**secrets['test_db'])
cursor = conn.cursor()

cursor.execute('create database if not exists biodb')

c:\python35\lib\site-packages\pymysql\cursors.py:166: Warning: (1007, "Can't create database 'biodb'; database exists")
  result = self._query(query)


1

In [7]:
#load data to dataframe
df = pd.read_csv(pathTofile,sep ='\t')
df.head()

,Gene ID,Gene Name,Design Element,g1_g2.p-value,g1_g2.t-statistic,g1_g2.log2foldchange
0,MIMAT0000062,MIMAT0000062,A_25_P00010086,0.702506,0.625362,0.078492
1,MIMAT0000062,MIMAT0000062,A_25_P00011584,0.975514,0.065110,0.007410
2,MIMAT0000063,MIMAT0000063,A_25_P00010070,0.139014,2.431636,0.556112
3,MIMAT0000063,MIMAT0000063,A_25_P00010071,0.049728,3.254492,0.811621
4,MIMAT0000064,MIMAT0000064,A_25_P00010072,0.098283,2.697813,0.757066


In [10]:
#create table from data frame

cursor.execute('use biodb')

#fix column names
df.columns = [re.sub('[ .-]','',x) for x in df.columns]
df.to_sql('dea',conn,flavor='mysql')

c:\python35\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


In [11]:
#check if table is created
cursor.execute("show tables")
cursor.fetchall()

(('ache',),
 ('dea',),
 ('drugbank',),
 ('temp',),
 ('temp1',),
 ('temp12',),
 ('uniprot',))

### Exercises

* write a query to fetch the first 10 lines from dea
* Count the number of rows in the table
* Calculat the average p_value for the entire data
* Calculate average p_value for each gene
* Find the gene with highest fold change, lowest fold change
* add your notebook to your GitHub